## Anotações Tech Challenge

link: [http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01]

O intuito desse markdown é fazer anotações sobre o formato e inconsistências encontradas nos arquivos para servir de documentação e de guia para o tratamento dos dados

In [10]:
import pandas as pd
import glob
import os

In [11]:
def fn_consolida_arquivos(caminho, separador=';'):
    lst_arq_import = glob.glob(caminho)
    df_lista = []
    for f in lst_arq_import:
        df_tmp = pd.read_csv(f, sep=separador)
        df_tmp['nomeArquivo'] = os.path.basename(f)
        df_lista.append(df_tmp)
    return pd.concat(df_lista)

# Arquivo Produção.csv

- 51 linhas: descrição dos produtos
- 55 colunas: id/nome do produto e cada coluna com o volume produção por ano (1970-2022)


Inconsistências:
- Algumas linhas correspondem aos agrupamentos dos produtos, por exemplo id 1 (VINHO DE MESA), é o agrupamento dos ids 2 até 4, mesmo comportamento acontece nos IDs 5, 9 e 14
- Não temos valor para SUCOS entre 73 e 84, nesse mesmo período o valor de produção do mosto aumentou muito, como o mosto faz parte do processo de fabricação da uva, creio que acabou sendo contabilizado tudo junto. Acontece algo parecido com os ids 39 e 42
    "O mosto corresponde ao líquido extraído da uva, enquanto o suco de uva representa o mosto processado, isto é, submetido a determinadas práticas para auxiliar na extração da matéria corante e garantir a sua clarificação, estabilização e conservação até o momento do consumo." -> https://ainfo.cnptia.embrapa.br/digital/bitstream/item/198428/1/Metodologia-Analise-Mosto-Suco-Uva-ed01-2010.pdf

In [12]:
df_prod = pd.read_csv(r".\Arquivos\Producao.csv", sep=";")
#Excluir coluna id do arquivo
df_prod.drop(columns=['id'], inplace=True)

#Criar campo categoria
df_prod['categoria'] = ''
df_prod.loc[1:3,'categoria'] = 'VINHO DE MESA'
df_prod.loc[5:7,'categoria'] = 'VINHO FINO DE MESA (VINÍFERA)'
df_prod.loc[9:13,'categoria'] = 'SUCO'
df_prod.loc[15:,'categoria'] = 'DERIVADOS'

#Excluir linhas com valores acumulados
df_prod_filtro = df_prod.query("produto not in ('VINHO DE MESA','VINHO FINO DE MESA (VINÍFERA)','SUCO','DERIVADOS')")

#Transpor colunas de anos em linhas
df_prod_melt = df_prod_filtro.melt(id_vars=['produto','categoria'], var_name='Ano', value_name='quantidade_L')
df_prod_melt

,produto,categoria,Ano,quantidade_L
0,Tinto,VINHO DE MESA,1970,174224052
1,Branco,VINHO DE MESA,1970,748400
2,Rosado,VINHO DE MESA,1970,42236152
3,Tinto,VINHO FINO DE MESA (VINÍFERA),1970,7591557
4,Branco,VINHO FINO DE MESA (VINÍFERA),1970,15562889
...,...,...,...,...
2486,Espumante orgânico,DERIVADOS,2022,0
2487,Destilado alcoólico simples de bagaceira,DERIVADOS,2022,4300
2488,Vinho acidificado,DERIVADOS,2022,0
2489,Mosto parcialmente fermentado,DERIVADOS,2022,0


### Arquivos Importação

In [13]:
df = fn_consolida_arquivos(r".\Arquivos\Imp*.csv")

In [14]:
#Ajustar colunas
dic_cat = {'ImpEspumantes.csv':'Espumantes', 'ImpFrescas.csv':'Uvas frescas', 'ImpPassas.csv':'Uvas passas','ImpSuco.csv':'Suco', 'ImpVinhos.csv':'Vinhos de mesa'}

df['2022'].fillna(df['2021.2'], inplace=True)
df['2022.1'].fillna(df['2021.3'], inplace=True)
df['Categoria'] = df['nomeArquivo'].map(dic_cat)
df.drop(columns=['Id','nomeArquivo','2021.2','2021.3'], inplace=True)

In [15]:
#Selecionar colunas de ano
lst_cols = [c for c in df.columns if c not in ['País','Categoria']]
#Renomear colunas de ano
lst_cols_rename = [c.replace('.1','')+'_Valor' if '.' in c else c+'_Peso' for c in lst_cols]
#Renomear df
df.columns = [c if c not in lst_cols else lst_cols_rename[lst_cols.index(c)] for c in df.columns]


In [16]:
df_melt = df.melt(id_vars=['País','Categoria'], var_name='Ano_Tipo', value_name='Valor')
df_melt['Valor'].fillna(0, inplace=True)
df_melt[['Ano','Tipo']] = df_melt['Ano_Tipo'].str.split("_", expand=True)

In [17]:
df_melt

,País,Categoria,Ano_Tipo,Valor,Ano,Tipo
0,Africa do Sul,Espumantes,1970_Peso,0.0,1970,Peso
1,Alemanha,Espumantes,1970_Peso,0.0,1970,Peso
2,Argentina,Espumantes,1970_Peso,4980.0,1970,Peso
3,Austrália,Espumantes,1970_Peso,0.0,1970,Peso
4,Austria,Espumantes,1970_Peso,0.0,1970,Peso
...,...,...,...,...,...,...
20559,Ucrânia,Vinhos de mesa,2022_Valor,0.0,2022,Valor
20560,Uruguai,Vinhos de mesa,2022_Valor,9867675.0,2022,Valor
20561,Não consta na tabela,Vinhos de mesa,2022_Valor,0.0,2022,Valor
20562,Não declarados,Vinhos de mesa,2022_Valor,0.0,2022,Valor


### Arquivos Exportação

In [24]:
df_exp = fn_consolida_arquivos(r".\Arquivos\Exp*.csv")

In [26]:
#Ajustar colunas
dic_cat = {'ExpEspumantes.csv':'Espumantes', 'ExpUva.csv':'Uvas frescas', 'ExpSuco.csv':'Suco', 'ExpVinho.csv':'Vinhos de mesa'}
df_exp['Categoria'] = df_exp['nomeArquivo'].map(dic_cat)
df_exp.drop(columns=['Id','nomeArquivo'], inplace=True)
df_exp

,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,1974,...,2018.1,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,Categoria
0,Alemanha,0,0,0,0,0,0,0,0,0,...,21373,1003,5466,2388,14767,142,265,1164,6560,Espumantes
1,Angola,0,0,0,0,0,0,0,0,0,...,280,1007,3615,24,38,0,0,26383,141588,Espumantes
2,Antigua e Barbuda,0,0,0,0,0,0,0,0,0,...,0,7,34,32,328,10,82,65,146,Espumantes
3,Antilhas Holandesas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Espumantes
4,Argentina,0,0,0,0,0,0,0,0,0,...,17243,0,0,315,894,0,0,0,0,Espumantes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Tuvalu,0,0,0,0,0,0,0,0,0,...,0,0,0,2,4,0,0,0,0,Vinhos de mesa
124,Uruguai,0,0,0,0,0,0,0,0,0,...,29617,6180,18497,0,0,136774,149842,637117,997367,Vinhos de mesa
125,Vanuatu,0,0,0,0,0,0,0,0,0,...,0,0,0,18,31,0,0,0,0,Vinhos de mesa
126,Venezuela,0,0,3200,2000,1350,750,11700,6458,9830,...,355,0,0,4086,9808,26415,35944,23220,32351,Vinhos de mesa


In [27]:
#Selecionar colunas de ano
lst_cols = [c for c in df_exp.columns if c not in ['País','Categoria']]
#Renomear colunas de ano
lst_cols_rename = [c.replace('.1','')+'_Valor' if '.' in c else c+'_Peso' for c in lst_cols]
#Renomear df
df_exp.columns = [c if c not in lst_cols else lst_cols_rename[lst_cols.index(c)] for c in df_exp.columns]

df_exp_melt = df.melt(id_vars=['País','Categoria'], var_name='Ano_Tipo', value_name='Valor')
df_exp_melt['Valor'].fillna(0, inplace=True)
df_exp_melt[['Ano','Tipo']] = df_exp_melt['Ano_Tipo'].str.split("_", expand=True)
df_melt

,País,Categoria,Ano_Tipo,Valor,Ano,Tipo
0,Africa do Sul,Espumantes,1970_Peso,0.0,1970,Peso
1,Alemanha,Espumantes,1970_Peso,0.0,1970,Peso
2,Argentina,Espumantes,1970_Peso,4980.0,1970,Peso
3,Austrália,Espumantes,1970_Peso,0.0,1970,Peso
4,Austria,Espumantes,1970_Peso,0.0,1970,Peso
...,...,...,...,...,...,...
20559,Ucrânia,Vinhos de mesa,2022_Valor,0.0,2022,Valor
20560,Uruguai,Vinhos de mesa,2022_Valor,9867675.0,2022,Valor
20561,Não consta na tabela,Vinhos de mesa,2022_Valor,0.0,2022,Valor
20562,Não declarados,Vinhos de mesa,2022_Valor,0.0,2022,Valor
